# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096801


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:44,  1.59s/it]

Rendering models:  10%|█         | 3/30 [00:06<00:52,  1.96s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:35,  1.42s/it]

Rendering models:  20%|██        | 6/30 [00:08<00:38,  1.59s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:13<00:01,  2.97it/s]

Rendering models:  90%|█████████ | 27/30 [00:14<00:01,  2.85it/s]

Rendering models:  93%|█████████▎| 28/30 [00:14<00:00,  3.24it/s]

Rendering models:  97%|█████████▋| 29/30 [00:14<00:00,  3.52it/s]

Jodelle                               0.000955
Jnursssmith5263                       0.001715
Idax98                                0.000445
Quarkgc                               0.000383
not-logged-in-42aeec5653762437f6b7    0.003591
awright5                              0.000604
AaronWallage117                       0.002123
jaksonA                               0.132137
not-logged-in-ef256f13ca4c9a6bfcf7    0.016567
not-logged-in-4c0d24701eeb1bad5fb6    0.000347
Jessica_Korkmaz                       0.003082
not-logged-in-5f30b2bb9b7f9d49da41    0.030587
nora.zoo                              0.000428
pangeli5                              0.002479
TRISTANM18                            0.000609
RaulE                                 0.000339
BraxtonKinney                         0.000422
not-logged-in-300dca7f45d1a5d1dd58    0.002391
not-logged-in-5d503e3b0aa7cdbe336e    0.000399
Lavadude                              0.070466
TateNachampassak                      0.001294
equidad1     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())